In [1]:
#|default_exp carmen_backend

In [2]:
#hide
!pip install -Uqq --upgrade --force-reinstall langchain langchain-openai
!pip install -Uqq langchain_experimental

In [3]:
#|export
import os
from langchain_openai import ChatOpenAI
import json

from langchain_core.tools import tool
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model = "gpt-4"

GAME_STATE_FILE = "./carmen_game_state.json"

In [4]:
#|export
def get_game_states():
    if not os.path.exists(GAME_STATE_FILE):
        return {}
    
    with open(GAME_STATE_FILE, "r") as f:
        return json.load(f)

In [5]:
#|export
def store_game_state(game_state: dict):
    game_states = get_game_states()
    game_states[game_state["case_id"]] = game_state

    with open(GAME_STATE_FILE, "w") as f:
        json.dump(game_states, f)

In [6]:
#|export
def get_game_state(case_id: str):
    return get_game_states()[case_id]

In [7]:
get_game_states()

{'8d354a22-4ee1-4eb5-a594-aa6a60a2d6c5': {'case_id': '8d354a22-4ee1-4eb5-a594-aa6a60a2d6c5',
  'suspect_name': 'Picasso Peculiar',
  'current_city': 'Chennai',
  'stolen_item': "The Louvre's Laughing Mona Lisa",
  'hops': ['Paris', 'Rome', 'Cairo', 'Tokyo', 'New York', 'Sydney'],
  'next_hop': 3}}

In [8]:
store_game_state(
    {
        "case_id": 1,
        "b": 2,
    }
)

In [9]:
!cat {GAME_STATE_FILE}

{"8d354a22-4ee1-4eb5-a594-aa6a60a2d6c5": {"case_id": "8d354a22-4ee1-4eb5-a594-aa6a60a2d6c5", "suspect_name": "Picasso Peculiar", "current_city": "Chennai", "stolen_item": "The Louvre's Laughing Mona Lisa", "hops": ["Paris", "Rome", "Cairo", "Tokyo", "New York", "Sydney"], "next_hop": 3}, "1": {"case_id": 1, "b": 2}}

In [10]:
!rm {GAME_STATE_FILE}

In [11]:
#|export
def new_game():
    new_game_prompt = """
    System: You are the game master for a detective game like "where in the world is Carmen San Diego". 
    To start the game, generate the following and return it as a json hash. 
    1. A random uuid, key: case_id
    2. A suspect name: suspect_name
    3. A source city, key: current_city 
    4. A famous artifact from that city, key: stolen_item
    5. A list of 6 unique cities with the current city as the first element, key: hops
    6. Set a field called next_hop to 1
    
    Make the suspect name and artifacts as campy as possible but use real city names. 

    Here is an example:
    {
        case_id: <uuid>,
        suspect_name: <suspect_name>,
        stolen_item: <item name>,
        hops: [
            <city 1>,
            <city 2>,
            <city 3>,
            <city 4>,
            <city 5>,
            <city 6>,
        ],
        next_hop: 1
    Output only the json hash. Do not provide any other text output. 
    """
    llm = ChatOpenAI(api_key=OPENAI_API_KEY, model=model)

    return llm.invoke(new_game_prompt).content
    

In [12]:
game_state = json.loads(new_game())
game_state

{'case_id': '123e4567-e89b-12d3-a456-426614174000',
 'suspect_name': 'Bumbling Boris',
 'current_city': 'Paris',
 'stolen_item': 'The Faux Mona Lisa',
 'hops': ['Paris', 'London', 'Berlin', 'Cairo', 'Tokyo', 'New York'],
 'next_hop': 1}

In [13]:
store_game_state(game_state)

In [14]:
!cat {GAME_STATE_FILE}

{"123e4567-e89b-12d3-a456-426614174000": {"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Bumbling Boris", "current_city": "Paris", "stolen_item": "The Faux Mona Lisa", "hops": ["Paris", "London", "Berlin", "Cairo", "Tokyo", "New York"], "next_hop": 1}}

In [15]:
case_id, _ = next(iter(get_game_states().items()))
case_id

'123e4567-e89b-12d3-a456-426614174000'

In [16]:
#|export
@tool
def fetch_game_state(case_id: str) -> str:
    """ Fetch the game state given the case_id """
    return json.dumps(get_game_states()[case_id])

In [17]:
fetch_game_state.invoke({"case_id": case_id})

'{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Bumbling Boris", "current_city": "Paris", "stolen_item": "The Faux Mona Lisa", "hops": ["Paris", "London", "Berlin", "Cairo", "Tokyo", "New York"], "next_hop": 1}'

In [18]:
#|export
@tool
def set_current_city(case_id: str, current_city: str):
    """ Sets the current city for the given case_id """
    game_state = get_game_state(case_id)
    game_state["current_city"] = current_city

    store_game_state(game_state)
    

In [19]:
set_current_city.invoke(
    {
        "case_id": case_id,
        "current_city": "Mumbai"
    }
)

get_game_state(case_id)

{'case_id': '123e4567-e89b-12d3-a456-426614174000',
 'suspect_name': 'Bumbling Boris',
 'current_city': 'Mumbai',
 'stolen_item': 'The Faux Mona Lisa',
 'hops': ['Paris', 'London', 'Berlin', 'Cairo', 'Tokyo', 'New York'],
 'next_hop': 1}

In [20]:
#|export
@tool
def update_game_state(case_id, key, value):
    """ Updates the game state based on the values given """
    game_state = get_game_state(case_id)
    game_state[key] = value

    store_game_state(game_state)

In [21]:
update_game_state.invoke(
    {
        "case_id": case_id,
        "key": "next_hop",
        "value": 3,
    }
)

get_game_state(case_id)

{'case_id': '123e4567-e89b-12d3-a456-426614174000',
 'suspect_name': 'Bumbling Boris',
 'current_city': 'Mumbai',
 'stolen_item': 'The Faux Mona Lisa',
 'hops': ['Paris', 'London', 'Berlin', 'Cairo', 'Tokyo', 'New York'],
 'next_hop': 3}

In [22]:
#|export
@tool
def generate_destinations(previous_city: str, next_city: str, current_city: str, exclude_list: str) -> str:
    """ Generates destinations for the player to travel to given the current city, previous city, 
    the next city, and a list of cities to exclude """
    llm = ChatOpenAI(api_key=OPENAI_API_KEY, model=model)

    prompt = f"""
        Generate a list of 4 cities. The list must include {previous_city}, {next_city} 
        and 2 other cities. It must not include {current_city} or any of the
        cities in {exclude_list}. Then shuffle the list before returning it. 
        
        Return this as a json list named destinations. Do not include any additional text.

        Here is an example of the json format:
        destinations: [
            <city 1>,
            <city 2>,
            <city 3>,
            <city 4>,
        ]
        """

    return llm.invoke(prompt).content

In [23]:
generate_destinations.invoke(
    {
        "previous_city": "London",
        "next_city": "Mumbai",
        "current_city": "Tokyo",
        "exclude_list": "Paris,Lima"
    }
)

'"destinations": [\n    "London",\n    "Mumbai",\n    "New York",\n    "Sydney"\n]'

In [24]:
#|export
@tool
def generate_regular_clues(city: str) -> str:
    """ Generate regular clues given a location """
    prompt = f"""
        Generate 3 clues. To generate each clue, do the following 
        1. Pick 3 locations at random out of bank, embassy, restaurant, tour guide, library. 
        2. Generate a clue for each of the locations
        
        The clue for a bank should be the currency of the country. For example if the city is London, 
        the clue should say "The suspect changed their currency to Pounds"
        
        The clue for the embassy should be about the flag of the country. For example, if the city is 
        New York, the clue should say "The suspect enquired about visiting a country with a red, white, and blue flag"
        
        The clue for the restaurant should be a good from the country the city is in. For example, for Mumbai, the clue
        should say "The suspect was looking for fresh samosas"
        
        The clue for tour guide should be something about a famous tourist attraction from the city. For example, for 
        Lima the clue should say "The suspect was interested in exploring Incan ruins"
        
        The clue for the library should be about a famous author from the country the city is in. For example, for London
        the clue could say "The suspect was interested in visiting the grave of Charles Dickens"
        
        Return it as a json hash with the following keys as a response:
        1. clues: a list of json hashes with keys location and clue
        
        Here is an example of the output json format:
        "clues": [
            {{
                "location": <location 1>,
                "clue": <clue 1>,
            }},
            {{
                "location": <location 2>,
                "clue": <clue 2>,
            }},
            {{
                "location": <location 3>,
                "clue": <clue 3>,
            }},
        ]
        
        Do not generate any additional text.
       
        Now generate clues for {city}
    """
    return ChatOpenAI(api_key=OPENAI_API_KEY, model=model).invoke(prompt).content

In [25]:
json.loads(generate_regular_clues.invoke(
    {
        "city": "Bangalore" 
    }
))

{'clues': [{'location': 'bank',
   'clue': 'The suspect changed their currency to Indian Rupees'},
  {'location': 'restaurant',
   'clue': 'The suspect was looking for fresh dosas'},
  {'location': 'library',
   'clue': 'The suspect was interested in the works of R.K. Narayan'}]}

In [26]:
#|export
@tool
def generate_mistaken_clues() -> str:
    """ Generate mistaken clues """
    prompt = f"""
        Generate 3 clues. To generate each clue, do the following 
        1. Pick 3 locations at random out of bank, embassy, restaurant, tour guide, library. 
        2. Set the clue to "No one with the suspect's description was seen here"
        
        Return it as a json hash with the following keys as a response:
            a. clues: a list of json hashes with keys location and clue

        Here is an example of the output json format:
        "clues": [
            {{
                "location": <location 1>,
                "clue": <clue 1>,
            }},
            {{
                "location": <location 2>,
                "clue": <clue 2>,
            }},
            {{
                "location": <location 3>,
                "clue": <clue 3>,
            }},
        ]
                
        Do not generate any additional text.

        Now generate the clues
    """
    return ChatOpenAI(api_key=OPENAI_API_KEY, model=model).invoke(prompt).content

In [27]:
json.loads(generate_mistaken_clues.invoke({}))

{'clues': [{'location': 'bank',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'embassy',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'restaurant',
   'clue': "No one with the suspect's description was seen here"}]}

In [27]:
#|export
@tool
def generate_arrest_clues(suspect_name: str) -> str:
    """ Generate arrest clues given the name of the suspect"""
    prompt = f"""
        Generate 3 clues. To generate each clue, do the following 
        1. Pick 3 locations at random out of bank, embassy, restaurant, tour guide, library. 
        2. For two of them set the clue to "Watch your step. You are getting close"
        3. For the third, set the clue to "Congratulations! You have arrested the suspect, {suspect_name}
        
        Return it as a json hash with the following keys as a response:
            a. clues: a list of json hashes with keys location and clue

        Here is an example of the output json format:
        "clues": [
            {{
                "location": <location 1>,
                "clue": <clue 1>,
            }},
            {{
                "location": <location 2>,
                "clue": <clue 2>,
            }},
            {{
                "location": <location 3>,
                "clue": <clue 3>,
            }},
        ]
 
        Do not generate any additional text.

        Now generate the clues for {suspect_name} 
    """
    return ChatOpenAI(api_key=OPENAI_API_KEY, model=model).invoke(prompt).content

In [28]:
json.loads(generate_arrest_clues.invoke(
    {
        "suspect_name": "Stealy McStealy"
    }
))

{'clues': [{'location': 'restaurant',
   'clue': 'Watch your step. You are getting close'},
  {'location': 'embassy', 'clue': 'Watch your step. You are getting close'},
  {'location': 'bank',
   'clue': 'Congratulations! You have arrested the suspect, Stealy McStealy'}]}

In [29]:
#|export
python_repl = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=PythonREPL().run,
)

In [30]:
#|export
tools = [
    fetch_game_state, 
    generate_destinations, 
    generate_regular_clues, 
    generate_arrest_clues, 
    generate_mistaken_clues,
    python_repl,
    update_game_state,   
]

In [31]:
#|export
def get_agent_executor():
    prompt = ChatPromptTemplate.from_messages(
        [
          ("system", "You are the game master for a detective game like 'where in the world is Carmen San Diego'. You will have to make a lot of decisions so look at the clues carefully."),
          ("placeholder", "{chat_history}"),
          ("human", "{input}"),
          ("placeholder", "{agent_scratchpad}"),
        ]
    )

    llm = ChatOpenAI(api_key=OPENAI_API_KEY, model=model)
    # Construct the tool calling agent
    agent = create_tool_calling_agent(llm, tools, prompt)

    # Create an agent executor by passing in the agent and tools
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    return agent_executor

In [32]:
#|export
def get_destinations(case_id):
    agent_executor = get_agent_executor()
    
    prompt = f"""
    Do the following:
    1. Fetch the game state for case_id {case_id}
    2. Write python code passing in the game state to do the following:
        a. Extract the current_city from the game state obtained above
        b. Get the city at the index next_hop from the hops list in the game state and save it as the next city
        c. If next_hop is 1 then set the previous city to "" otherwise get the city at the index next_hop -1 from 
            the hops list in the game state and save it as the previous city
        d. Add all other cities in the hops list to an exclude list
    
    3. Based on the output above, generate 4 destinations for the player to travel to including the previous city,
    current city, next city, and cities not in the exclude list. 
    4. Shuffle the list
    
    Return the output as a json hash with the following keys:
    1. city: The current city
    2. destinations: a list of the cities

    Here is an example of the output format:
    {{
        "city": <current city>
        "destinations": [
            <city 1>,
            <city 2>,
            <city 3>,
            <city 4>,
        ]
    }}
            
    Generate no additional text 
    """
    ret = agent_executor.invoke(
        {
            "input": prompt
        }
    )

    return ret['output']

In [33]:
json.loads(get_destinations(case_id))



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Bumbling Boris", "current_city": "Mumbai", "stolen_item": "The Faux Mona Lisa", "hops": ["Paris", "London", "Berlin", "Cairo", "Tokyo", "New York"], "next_hop": 3}

Python REPL can execute arbitrary code. Use with caution.



Invoking: `python_repl` with `game_state = {"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Bumbling Boris", "current_city": "Mumbai", "stolen_item": "The Faux Mona Lisa", "hops": ["Paris", "London", "Berlin", "Cairo", "Tokyo", "New York"], "next_hop": 3}
current_city = game_state['current_city']
next_hop = game_state['next_hop']
next_city = game_state['hops'][next_hop]
previous_city = '' if next_hop == 1 else game_state['hops'][next_hop - 1]
exclude_list = game_state['hops']
print(current_city, next_city, previous_city, exclude_list)`


Mumbai Cairo Berlin ['Paris', 'London', 'Berlin', 'Cairo', 'Tokyo', 'New York']

Invoking: `generate_destinations` with `{'previous_city': 'Berlin', 'next_city': 'Cairo', 'current_city': 'Mumbai', 'exclude_list': "['Paris', 'London', 'Berlin', 'Cairo', 'Tokyo', 'New York']"}`


The task seems to be contradicting itself. It first asks to include Berlin and Cairo in the list, but then it says not to include any cities from the list t

{'city': 'Mumbai', 'destinations': ['Berlin', 'Cairo', 'Madrid', 'Sydney']}

In [34]:
#|export
def get_clues(case_id):
    agent_executor = get_agent_executor()
    
    prompt = f"""
    1. Fetch the game state for case_id {case_id}.
    2. Write and execute python code passing in the game state to do the following to figure out what kind of 
       clue to generate:
        a. Extract next_hop from the game state
        b. If next_hop is equal to 5 only then print "arrest clues" as the type of clues
        c. Else If the current city is not in the hops list only then print "mistaken clues" as the type of clue
        d. Else print "regular clues" as the type of clue
    3. Based on the output of the code generate the right type of clue for the city at index next_hop in the hops list
    
    Return the output exactly as a hash encoded in json returned from the tool. Generate no additional text
    """
    ret = agent_executor.invoke(
        {
            "input": prompt
        }
    )

    return ret['output']

In [37]:
json.loads(get_clues(case_id))



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '8d354a22-4ee1-4eb5-a594-aa6a60a2d6c5'}`


{"case_id": "8d354a22-4ee1-4eb5-a594-aa6a60a2d6c5", "suspect_name": "Picasso Peculiar", "current_city": "Mumbai", "stolen_item": "The Louvre's Laughing Mona Lisa", "hops": ["Paris", "Rome", "Cairo", "Tokyo", "New York", "Sydney"], "next_hop": 3}
Invoking: `python_repl` with `game_state = {'case_id': '8d354a22-4ee1-4eb5-a594-aa6a60a2d6c5', 'suspect_name': 'Picasso Peculiar', 'current_city': 'Mumbai', 'stolen_item': 'The Louvre's Laughing Mona Lisa', 'hops': ['Paris', 'Rome', 'Cairo', 'Tokyo', 'New York', 'Sydney'], 'next_hop': 3}
next_hop = game_state['next_hop']
if next_hop == 5:
    print('arrest clues')
elif game_state['current_city'] not in game_state['hops']:
    print('mistaken clues')
else:
    print('regular clues')`


SyntaxError('unterminated string literal (detected at line 1)', ('<string>', 1, 254, "game_state = {'case_id': '8d354a22-4ee1-4eb5-a5

{'clues': [{'location': 'bank',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'restaurant',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'library',
   'clue': "No one with the suspect's description was seen here"}]}

In [37]:
cur_city = get_game_state(case_id)["current_city"]
set_current_city.invoke(
    {
        "case_id": case_id,
        "current_city": "Pune"
    }
)
json.loads(get_clues(case_id))
set_current_city.invoke(
    {
        "case_id": case_id,
        "current_city": cur_city
    }
)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Bumbling Boris", "current_city": "Pune", "stolen_item": "The Faux Mona Lisa", "hops": ["Paris", "London", "Berlin", "Cairo", "Tokyo", "New York"], "next_hop": 3}
Invoking: `python_repl` with `game_state = {"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Bumbling Boris", "current_city": "Pune", "stolen_item": "The Faux Mona Lisa", "hops": ["Paris", "London", "Berlin", "Cairo", "Tokyo", "New York"], "next_hop": 3}
next_hop = game_state['next_hop']
if next_hop == 5:
    print("arrest clues")
elif game_state['current_city'] not in game_state['hops']:
    print("mistaken clues")
else:
    print("regular clues")`


mistaken clues

Invoking: `generate_mistaken_clues` with `{}`


{
"clues": [
    {
        "location": "embassy",
        "clue": "No one with the suspect's descri

In [40]:
nxt_hop = get_game_state(case_id)["next_hop"]
update_game_state.invoke(
    {
        "case_id": case_id,
        "key": "next_hop",
        "value": 5,
    }
)

json.loads(get_clues(case_id))

update_game_state.invoke(
    {
        "case_id": case_id,
        "key": "next_hop",
        "value": nxt_hop,
    }
)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Bumbling Boris", "current_city": "Mumbai", "stolen_item": "The Faux Mona Lisa", "hops": ["Paris", "London", "Berlin", "Cairo", "Tokyo", "New York"], "next_hop": 5}
Invoking: `python_repl` with `game_state = {"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Bumbling Boris", "current_city": "Mumbai", "stolen_item": "The Faux Mona Lisa", "hops": ["Paris", "London", "Berlin", "Cairo", "Tokyo", "New York"], "next_hop": 5}
next_hop = game_state['next_hop']
current_city = game_state['current_city']
hops = game_state['hops']
if next_hop == 5:
    print('arrest clues')
elif current_city not in hops:
    print('mistaken clues')
else:
    print('regular clues')`


arrest clues

Invoking: `generate_arrest_clues` with `{'suspect_name': 'Bumbling Boris'}`


{
    "clues": [
        {


In [41]:
#|export
def travel(case_id, city):
    agent_executor = get_agent_executor()
    
    prompt = f"""
        1. Fetch the game state for case_id {case_id}
        2. Write python code passing in the game state to do the following:
            a. If {city} is not in the hops list, update current_city to {city} and print out the 
            new values of current_city and next_hop as a json hash
            b. Else if {city} is in the hops list and the index of {city} is <= next_hop then 
            set next_hop to the index + 1 and current_city to {city}. Print out the 
            new values of current_city and next_hop as a json hash
            c. Else if {city} is in hops list and the index of {city} is greater than next_hop 
            print the error "Error: You cannot travel to that city"

        If there is no error from the python code, 
        1. Update the current_city and next_hop in the game state
        2. return the current city as json hash with a current_city key. 
        
        Else if there is an error from the python code, return the error "You cannot travel to that city" as a 
        json hash with the key "error".
        
        Here is an example of the output format when there is no error:
        {{
            "current_city": <city>
        }}

        Here is an example of the output format when there is an error:
        {{
            "error": "You cannot travel to that city"
        }}

        Generate no additional text.

    """

    ret = agent_executor.invoke(
        {
            "input": prompt
        }
    )

    return ret['output']
    

In [42]:
json.loads(travel(case_id, "New York"))



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Bumbling Boris", "current_city": "Mumbai", "stolen_item": "The Faux Mona Lisa", "hops": ["Paris", "London", "Berlin", "Cairo", "Tokyo", "New York"], "next_hop": 3}
Invoking: `python_repl` with `import json

# Provided game state
state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Bumbling Boris', 'current_city': 'Mumbai', 'stolen_item': 'The Faux Mona Lisa', 'hops': ['Paris', 'London', 'Berlin', 'Cairo', 'Tokyo', 'New York'], 'next_hop': 3}

# Desired city
new_city = 'New York'

if new_city not in state['hops']:
    state['current_city'] = new_city
    print(json.dumps({'current_city': state['current_city'], 'next_hop': state['next_hop']}))
else:
    new_city_index = state['hops'].index(new_city)
    if new_city_index <= state['next_hop']:
        state['next_hop']

{'error': 'You cannot travel to that city'}

In [43]:
get_game_state(case_id)

{'case_id': '123e4567-e89b-12d3-a456-426614174000',
 'suspect_name': 'Bumbling Boris',
 'current_city': 'Mumbai',
 'stolen_item': 'The Faux Mona Lisa',
 'hops': ['Paris', 'London', 'Berlin', 'Cairo', 'Tokyo', 'New York'],
 'next_hop': 3}

In [44]:
json.loads(travel(case_id, "Chennai"))
get_game_state(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Bumbling Boris", "current_city": "Mumbai", "stolen_item": "The Faux Mona Lisa", "hops": ["Paris", "London", "Berlin", "Cairo", "Tokyo", "New York"], "next_hop": 3}
Invoking: `python_repl` with `import json

# game state
state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Bumbling Boris', 'current_city': 'Mumbai', 'stolen_item': 'The Faux Mona Lisa', 'hops': ['Paris', 'London', 'Berlin', 'Cairo', 'Tokyo', 'New York'], 'next_hop': 3}

# check if Chennai is in the hops list
if 'Chennai' not in state['hops']:
    state['current_city'] = 'Chennai'
    result = {'current_city': state['current_city'], 'next_hop': state['next_hop']}
else:
    index = state['hops'].index('Chennai')
    if index <= state['next_hop']:
        state['next_hop'] = index + 1
        state['curre

{'case_id': '123e4567-e89b-12d3-a456-426614174000',
 'suspect_name': 'Bumbling Boris',
 'current_city': 'Chennai',
 'stolen_item': 'The Faux Mona Lisa',
 'hops': ['Paris', 'London', 'Berlin', 'Cairo', 'Tokyo', 'New York'],
 'next_hop': 3}

In [45]:
json.loads(get_clues(case_id))



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Bumbling Boris", "current_city": "Chennai", "stolen_item": "The Faux Mona Lisa", "hops": ["Paris", "London", "Berlin", "Cairo", "Tokyo", "New York"], "next_hop": 3}
Invoking: `python_repl` with `game_state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Bumbling Boris', 'current_city': 'Chennai', 'stolen_item': 'The Faux Mona Lisa', 'hops': ['Paris', 'London', 'Berlin', 'Cairo', 'Tokyo', 'New York'], 'next_hop': 3}
next_hop = game_state['next_hop']
if next_hop == 5:
    print('arrest clues')
elif game_state['current_city'] not in game_state['hops']:
    print('mistaken clues')
else:
    print('regular clues')`


mistaken clues

Invoking: `generate_mistaken_clues` with `{}`


{
    "clues": [
        {
            "location": "bank",
            "clue": "No one with t

{'clues': [{'location': 'bank',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'restaurant',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'library',
   'clue': "No one with the suspect's description was seen here"}]}

In [46]:
from nbdev.export import nb_export

nb_export('./carmen.ipynb', './')

In [47]:
!ls

carmen_backend.py  carmen_game_state.json  carmen.ipynb


In [37]:
game_state = json.loads(new_game())
game_states[game_state["case_id"]] = game_state
game_states

{'b7b4d3a7-1dd4-4b6b-8f8d-3f2db45b2a8f': {'case_id': 'b7b4d3a7-1dd4-4b6b-8f8d-3f2db45b2a8f',
  'suspect_name': 'Mysterio McVanish',
  'current_city': 'Berlin',
  'stolen_item': 'The Laughing Mona Lisa',
  'hops': ['Paris', 'London', 'Tokyo', 'New York', 'Sydney', 'Cairo'],
  'next_hop': 1},
 'c6e6e3e7-80e9-4a9a-8f8a-6c54e3c6b9de': {'case_id': 'c6e6e3e7-80e9-4a9a-8f8a-6c54e3c6b9de',
  'suspect_name': 'Mysterio McVanity',
  'current_city': 'Paris',
  'stolen_item': 'The Dazzling Eiffel Tower Keychain',
  'hops': ['Paris', 'London', 'New York', 'Cairo', 'Berlin', 'Tokyo'],
  'next_hop': 1}}

In [38]:
case_id, _ = next(iter(game_states.items()))

In [39]:
get_clues(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'b7b4d3a7-1dd4-4b6b-8f8d-3f2db45b2a8f'}`


{"case_id": "b7b4d3a7-1dd4-4b6b-8f8d-3f2db45b2a8f", "suspect_name": "Mysterio McVanish", "current_city": "Berlin", "stolen_item": "The Laughing Mona Lisa", "hops": ["Paris", "London", "Tokyo", "New York", "Sydney", "Cairo"], "next_hop": 1}
Invoking: `python_repl` with `game_state = {"case_id": "b7b4d3a7-1dd4-4b6b-8f8d-3f2db45b2a8f", "suspect_name": "Mysterio McVanish", "current_city": "Berlin", "stolen_item": "The Laughing Mona Lisa", "hops": ["Paris", "London", "Tokyo", "New York", "Sydney", "Cairo"], "next_hop": 1}
next_hop = game_state['next_hop']
current_city = game_state['current_city']
hops = game_state['hops']

if next_hop == 5:
    print('arrest clues')
elif current_city not in hops:
    print('mistaken clues')
else:
    print('regular clues')`


mistaken clues

Invoking: `generate_mistaken_clues` with `{}`


{
    "clues": [
        {
            "

'{"clues": [{"location": "bank", "clue": "No one with the suspect\'s description was seen here"}, {"location": "embassy", "clue": "No one with the suspect\'s description was seen here"}, {"location": "restaurant", "clue": "No one with the suspect\'s description was seen here"}]}'

In [40]:
travel(case_id, "New York")



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'b7b4d3a7-1dd4-4b6b-8f8d-3f2db45b2a8f'}`


{"case_id": "b7b4d3a7-1dd4-4b6b-8f8d-3f2db45b2a8f", "suspect_name": "Mysterio McVanish", "current_city": "Berlin", "stolen_item": "The Laughing Mona Lisa", "hops": ["Paris", "London", "Tokyo", "New York", "Sydney", "Cairo"], "next_hop": 1}

KeyboardInterrupt: 